In [23]:
import findspark
findspark.init('/home/ubuntu/spark-2.3.2-bin-hadoop2.7')
import pyspark

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('nlp_example').getOrCreate()

In [24]:
from pyspark.ml.feature import Tokenizer, RegexTokenizer

In [25]:
from pyspark.sql.functions import col, udf
from pyspark.sql.types import IntegerType

In [26]:
sentence_df = spark.createDataFrame([
        (0, 'Hi I heard about Spark'),
        (1, 'I wish Java could use case classes'),
        (2, 'Logistic,regression,models,are,neat')
    ], ['id', 'sentence'])

In [27]:
sentence_df.show()

+---+--------------------+
| id|            sentence|
+---+--------------------+
|  0|Hi I heard about ...|
|  1|I wish Java could...|
|  2|Logistic,regressi...|
+---+--------------------+



In [28]:
tokenizer = Tokenizer(inputCol='sentence', outputCol='words')

In [29]:
regex_tokenizer = RegexTokenizer(inputCol='sentence', outputCol='words', pattern='\\W')

In [30]:
count_tokens = udf(lambda words: len(words), IntegerType())

In [31]:
tokenized = tokenizer.transform(sentence_df)

In [32]:
tokenized.show()

+---+--------------------+--------------------+
| id|            sentence|               words|
+---+--------------------+--------------------+
|  0|Hi I heard about ...|[hi, i, heard, ab...|
|  1|I wish Java could...|[i, wish, java, c...|
|  2|Logistic,regressi...|[logistic,regress...|
+---+--------------------+--------------------+



In [33]:
tokenized.withColumn('tokens', count_tokens(col('words'))).show()

+---+--------------------+--------------------+------+
| id|            sentence|               words|tokens|
+---+--------------------+--------------------+------+
|  0|Hi I heard about ...|[hi, i, heard, ab...|     5|
|  1|I wish Java could...|[i, wish, java, c...|     7|
|  2|Logistic,regressi...|[logistic,regress...|     1|
+---+--------------------+--------------------+------+



In [34]:
rg_tokenized = regex_tokenizer.transform(sentence_df)

In [35]:
rg_tokenized.withColumn('tokens', count_tokens(col('words'))).show()

+---+--------------------+--------------------+------+
| id|            sentence|               words|tokens|
+---+--------------------+--------------------+------+
|  0|Hi I heard about ...|[hi, i, heard, ab...|     5|
|  1|I wish Java could...|[i, wish, java, c...|     7|
|  2|Logistic,regressi...|[logistic, regres...|     5|
+---+--------------------+--------------------+------+



In [36]:
from pyspark.ml.feature import StopWordsRemover

In [37]:
sentence_df2 = spark.createDataFrame([
        (0, ['I', 'saw', 'the', 'green', 'horse']),
        (1, ['Mary', 'had', 'a', 'little', 'lamb']),
    ], ['id', 'tokens'])

In [38]:
sentence_df2.show()

+---+--------------------+
| id|              tokens|
+---+--------------------+
|  0|[I, saw, the, gre...|
|  1|[Mary, had, a, li...|
+---+--------------------+



In [39]:
remover = StopWordsRemover(inputCol='tokens', outputCol='filtered')

In [40]:
remover.transform(sentence_df2).show()

+---+--------------------+--------------------+
| id|              tokens|            filtered|
+---+--------------------+--------------------+
|  0|[I, saw, the, gre...| [saw, green, horse]|
|  1|[Mary, had, a, li...|[Mary, little, lamb]|
+---+--------------------+--------------------+



In [41]:
from pyspark.ml.feature import NGram

In [44]:
word_df = spark.createDataFrame([
    (0, ["Hi", "I", "heard", "about", "Spark"]),
    (1, ["I", "wish", "Java", "could", "use", "case", "classes"]),
    (2, ["Logistic", "regression", "models", "are", "neat"])
], ["id", "words"])

In [45]:
ngram = NGram(n=2, inputCol='words', outputCol='grams')

In [46]:
ngram.transform(word_df).show()

+---+--------------------+--------------------+
| id|               words|               grams|
+---+--------------------+--------------------+
|  0|[Hi, I, heard, ab...|[Hi I, I heard, h...|
|  1|[I, wish, Java, c...|[I wish, wish Jav...|
|  2|[Logistic, regres...|[Logistic regress...|
+---+--------------------+--------------------+



In [47]:
ngram.transform(word_df).select('grams').show(truncate=False)

+------------------------------------------------------------------+
|grams                                                             |
+------------------------------------------------------------------+
|[Hi I, I heard, heard about, about Spark]                         |
|[I wish, wish Java, Java could, could use, use case, case classes]|
|[Logistic regression, regression models, models are, are neat]    |
+------------------------------------------------------------------+



In [48]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer

In [49]:
sentence_df3 = spark.createDataFrame([
    (0.0, "Hi I heard about Spark"),
    (0.0, "I wish Java could use case classes"),
    (1.0, "Logistic regression models are neat")
], ["label", "sentence"])

In [50]:
sentence_df3.show()

+-----+--------------------+
|label|            sentence|
+-----+--------------------+
|  0.0|Hi I heard about ...|
|  0.0|I wish Java could...|
|  1.0|Logistic regressi...|
+-----+--------------------+



In [51]:
tokenizer = Tokenizer(inputCol='sentence', outputCol='words')

In [52]:
words_data = tokenizer.transform(sentence_df3)

In [54]:
words_data.show(truncate=False)

+-----+-----------------------------------+------------------------------------------+
|label|sentence                           |words                                     |
+-----+-----------------------------------+------------------------------------------+
|0.0  |Hi I heard about Spark             |[hi, i, heard, about, spark]              |
|0.0  |I wish Java could use case classes |[i, wish, java, could, use, case, classes]|
|1.0  |Logistic regression models are neat|[logistic, regression, models, are, neat] |
+-----+-----------------------------------+------------------------------------------+



In [55]:
hashing_tf = HashingTF(inputCol='words', outputCol='raw_features')

In [57]:
featurized_data = hashing_tf.transform(words_data)

In [58]:
idf = IDF(inputCol='raw_features', outputCol='features')

In [59]:
idf_model = idf.fit(featurized_data).transform(featurized_data)

In [61]:
idf_model.select('label', 'features').show(truncate=False)

+-----+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|label|features                                                                                                                                                                                        |
+-----+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|0.0  |(262144,[24417,49304,73197,91137,234657],[0.28768207245178085,0.6931471805599453,0.6931471805599453,0.6931471805599453,0.6931471805599453])                                                     |
|0.0  |(262144,[20719,24417,55551,116873,147765,162369,192310],[0.6931471805599453,0.28768207245178085,0.6931471805599453,0.6931471805599453,0.6931471805599453,0.6931471805599453,0.693147180559945

In [62]:
from pyspark.ml.feature import CountVectorizer

In [63]:
letter_df = spark.createDataFrame([
    (0, "a b c".split(" ")),
    (1, "a b b c a".split(" "))
], ["id", "words"])

In [64]:
letter_df.show()

+---+---------------+
| id|          words|
+---+---------------+
|  0|      [a, b, c]|
|  1|[a, b, b, c, a]|
+---+---------------+



In [65]:
count_vectorizer = CountVectorizer(inputCol='words', outputCol='features', vocabSize=3, minDF=2.0)

In [66]:
model = count_vectorizer.fit(letter_df).transform(letter_df)

In [68]:
model.show(truncate=False)

+---+---------------+-------------------------+
|id |words          |features                 |
+---+---------------+-------------------------+
|0  |[a, b, c]      |(3,[0,1,2],[1.0,1.0,1.0])|
|1  |[a, b, b, c, a]|(3,[0,1,2],[2.0,2.0,1.0])|
+---+---------------+-------------------------+

